# Trabajo Práctico 3 - Regularización aplicada a la EPH 

## Gil Deza, Hüppi Lo Prete, Walker 

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error

In [ ]:
# Utilizamos el código de la clase tutorial para crear un botón para ocultar el código 
#Fuente: https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer

import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

### Parte I

#### 1) 

Las variables que pueden servir como predictores para entender si un hogar es pobre o no son:
        -Variables que representan las características de la vivienda como el acceso al agua, a un baño, materiales de construcción y si está dentro de un barrio de emergencia. 
        -Variables que representan las características habitacionales del hogar como por ejemplo, la cantidad de cuartos destinados para dormir, régimen de tenecia del terreno y qué combustible se utiliza para cocinar. 
        -Variables denominada "Estrategias del hogar", entre ellas se encuentran si cobran un subsidio, si tienen una beca de estudio, si tuvieron que vender sus pertenencias en el último tiempo,  menores de edad trabajando, cantidad de personas que viven en el hogar, decil de ingresos, entre otras. Consideramos que esta última variable es muy relevante dado que marca la diferencia entre medir la pobreza a nivel hogar o a nivel individuo debido a que hogares pobres tienden a tener más miembros.

#### 2)

In [2]:
# Abrimos la base de hogar 
df=pd.read_excel("usu_hogar_T122.xls")
df
# Abrimos la tabla de usuarios 
df_usu=pd.read_excel("usu_individual_T122.xls")

In [3]:
# Armamos un nuevo df solo con las observaciones de Buenos Aires y Gran Buenos Aires
df_ba = df.loc[df['AGLOMERADO'].isin([32, 33])]
df_ba


,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,REALIZADA,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,...,GDECCFR,PDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
11,TQRMNOQSYHMOTOCDEIJAH00698520,2022,1,1,1,1,S,33,4625,2,...,0.0,NaN,0,4410,1,0,98,0,0,0
23,TQRMNOPSSHKKMMCDEIIAD00780111,2022,1,1,1,1,S,32,803,1,...,10.0,NaN,8,1440,1,2,98,0,0,0
52,TQRMNORSUHLMNPCDEIIAD00718267,2022,1,1,1,1,S,32,2785,2,...,10.0,NaN,9,4073,2,0,98,0,0,0
61,TQRMNORUPHMLRMCDEIJAH00698717,2022,1,1,1,1,S,33,1698,1,...,12.0,NaN,12,0,2,1,3,0,0,0
72,TQRMNOTUYHMMKNCDEIJAH00698671,2022,1,1,1,1,S,33,3764,1,...,12.0,NaN,12,0,2,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16778,TQRMNOPXQHJOKPCDEIJAH00718715,2022,1,1,1,1,S,33,3278,1,...,1.0,NaN,1,5814,2,0,3,4,0,0
16779,TQRMNOPXRHJOKPCDEIJAH00718716,2022,1,1,1,1,S,33,3976,1,...,3.0,NaN,4,6427,2,1,3,4,0,0
16780,TQRMNOPQSHMMKPCDEIJAH00780780,2022,1,1,1,1,S,33,2607,1,...,3.0,NaN,4,5084,1,2,98,0,0,0
16781,TQRMNOPWPHMLLLCDEIJAH00780781,2022,1,1,1,1,S,33,2325,1,...,5.0,NaN,6,4528,1,2,98,0,0,0


#### 3)

In [8]:
df_ba_unido = pd.merge(df_ba, df_usu, on=("CODUSU","NRO_HOGAR"), how="inner", validate="one_to_many")
df_ba_unido
print(list(df_ba_unido.columns))

['CODUSU', 'ANO4_x', 'TRIMESTRE_x', 'NRO_HOGAR', 'REALIZADA', 'REGION_x', 'MAS_500_x', 'AGLOMERADO_x', 'PONDERA_x', 'IV1', 'IV1_ESP', 'IV2', 'IV3', 'IV3_ESP', 'IV4', 'IV5', 'IV6', 'IV7', 'IV7_ESP', 'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2', 'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3', 'II5', 'II5_1', 'II6', 'II6_1', 'II7', 'II7_ESP', 'II8', 'II8_ESP', 'II9', 'V1', 'V2', 'V21', 'V22', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A', 'V19_B', 'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10', 'ITF_x', 'DECIFR_x', 'IDECIFR_x', 'RDECIFR_x', 'GDECIFR_x', 'PDECIFR_x', 'ADECIFR_x', 'IPCF_x', 'DECCFR_x', 'IDECCFR_x', 'RDECCFR_x', 'GDECCFR_x', 'PDECCFR_x', 'ADECCFR_x', 'PONDIH_x', 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4', 'ANO4_y', 'TRIMESTRE_y', 'COMPONENTE', 'H15', 'REGION_y', 'MAS_500_y', 'AGLOMERADO_y', 'PONDERA_y', 'CH03', 'CH04', 'CH05', 'CH06', 'CH07', 'CH08', 'CH09', 'CH10', 'CH11', 'CH12', 

Eliminamos todas las columnas duplicadas luego del merge, dado que se encontraban en ambas bases.

In [15]:
# Observamos cuáles son las columnas que están repetidas, antes de eliminarlas:
df_variables_dup = df_ba_unido.filter(regex='_y')
df_variables_dup

# Las eliminamos
df_ba_unido.drop(columns=(df_ba_unido.filter(regex='_y')), inplace=True)

print(list(df_ba_unido.columns))

['CODUSU', 'ANO4_x', 'TRIMESTRE_x', 'NRO_HOGAR', 'REALIZADA', 'REGION_x', 'MAS_500_x', 'AGLOMERADO_x', 'PONDERA_x', 'IV1', 'IV1_ESP', 'IV2', 'IV3', 'IV3_ESP', 'IV4', 'IV5', 'IV6', 'IV7', 'IV7_ESP', 'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2', 'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3', 'II5', 'II5_1', 'II6', 'II6_1', 'II7', 'II7_ESP', 'II8', 'II8_ESP', 'II9', 'V1', 'V2', 'V21', 'V22', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A', 'V19_B', 'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10', 'ITF_x', 'DECIFR_x', 'IDECIFR_x', 'RDECIFR_x', 'GDECIFR_x', 'PDECIFR_x', 'ADECIFR_x', 'IPCF_x', 'DECCFR_x', 'IDECCFR_x', 'RDECCFR_x', 'GDECCFR_x', 'PDECCFR_x', 'ADECCFR_x', 'PONDIH_x', 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4', 'COMPONENTE', 'H15', 'CH03', 'CH04', 'CH05', 'CH06', 'CH07', 'CH08', 'CH09', 'CH10', 'CH11', 'CH12', 'CH13', 'CH14', 'CH15', 'CH15_COD', 'CH16', 'CH16_COD', 'NIVEL_ED', 'ESTADO', '

#### 4)

Utilizamos la función dropna del paquete de Pandas para eliminar columnas con observaciones vacias

#### 5)

In [16]:
# Limpiamos la base de los datos nulos
# Eliminamos columnas que tienen más del 50% de observaciones vacías 
df_ba_unido_limpio = df_ba_unido.dropna(axis=1, thresh=3372)
#df_ba_unido_limpio.to_excel("base2.xlsx", sheet_name="sheet1", index=False)

In [17]:
# Chequeamos que ya no quedan missings values
for i in range(len(df_ba_unido_limpio.index)) : 
    print("Número de missing values en la fila", i + 1, ":",
         df_ba_unido_limpio.iloc[i].isnull().sum())

Número de missing values en la fila 1 : 0
Número de missing values en la fila 2 : 0
Número de missing values en la fila 3 : 0
Número de missing values en la fila 4 : 0
Número de missing values en la fila 5 : 0
Número de missing values en la fila 6 : 0
Número de missing values en la fila 7 : 0
Número de missing values en la fila 8 : 0
Número de missing values en la fila 9 : 0
Número de missing values en la fila 10 : 0
Número de missing values en la fila 11 : 0
Número de missing values en la fila 12 : 0
Número de missing values en la fila 13 : 0
Número de missing values en la fila 14 : 0
Número de missing values en la fila 15 : 0
Número de missing values en la fila 16 : 0
Número de missing values en la fila 17 : 0
Número de missing values en la fila 18 : 0
Número de missing values en la fila 19 : 0
Número de missing values en la fila 20 : 0
Número de missing values en la fila 21 : 0
Número de missing values en la fila 22 : 0
Número de missing values en la fila 23 : 0
Número de missing va

Número de missing values en la fila 211 : 0
Número de missing values en la fila 212 : 0
Número de missing values en la fila 213 : 0
Número de missing values en la fila 214 : 0
Número de missing values en la fila 215 : 0
Número de missing values en la fila 216 : 0
Número de missing values en la fila 217 : 0
Número de missing values en la fila 218 : 0
Número de missing values en la fila 219 : 0
Número de missing values en la fila 220 : 0
Número de missing values en la fila 221 : 0
Número de missing values en la fila 222 : 0
Número de missing values en la fila 223 : 0
Número de missing values en la fila 224 : 0
Número de missing values en la fila 225 : 0
Número de missing values en la fila 226 : 0
Número de missing values en la fila 227 : 0
Número de missing values en la fila 228 : 0
Número de missing values en la fila 229 : 0
Número de missing values en la fila 230 : 0
Número de missing values en la fila 231 : 0
Número de missing values en la fila 232 : 0
Número de missing values en la f

Número de missing values en la fila 443 : 0
Número de missing values en la fila 444 : 0
Número de missing values en la fila 445 : 0
Número de missing values en la fila 446 : 0
Número de missing values en la fila 447 : 0
Número de missing values en la fila 448 : 0
Número de missing values en la fila 449 : 0
Número de missing values en la fila 450 : 0
Número de missing values en la fila 451 : 0
Número de missing values en la fila 452 : 0
Número de missing values en la fila 453 : 0
Número de missing values en la fila 454 : 0
Número de missing values en la fila 455 : 0
Número de missing values en la fila 456 : 0
Número de missing values en la fila 457 : 0
Número de missing values en la fila 458 : 0
Número de missing values en la fila 459 : 0
Número de missing values en la fila 460 : 0
Número de missing values en la fila 461 : 0
Número de missing values en la fila 462 : 0
Número de missing values en la fila 463 : 0
Número de missing values en la fila 464 : 0
Número de missing values en la f

Número de missing values en la fila 704 : 0
Número de missing values en la fila 705 : 0
Número de missing values en la fila 706 : 0
Número de missing values en la fila 707 : 0
Número de missing values en la fila 708 : 0
Número de missing values en la fila 709 : 0
Número de missing values en la fila 710 : 0
Número de missing values en la fila 711 : 0
Número de missing values en la fila 712 : 0
Número de missing values en la fila 713 : 0
Número de missing values en la fila 714 : 0
Número de missing values en la fila 715 : 0
Número de missing values en la fila 716 : 0
Número de missing values en la fila 717 : 0
Número de missing values en la fila 718 : 0
Número de missing values en la fila 719 : 0
Número de missing values en la fila 720 : 0
Número de missing values en la fila 721 : 0
Número de missing values en la fila 722 : 0
Número de missing values en la fila 723 : 0
Número de missing values en la fila 724 : 0
Número de missing values en la fila 725 : 0
Número de missing values en la f

Número de missing values en la fila 921 : 0
Número de missing values en la fila 922 : 0
Número de missing values en la fila 923 : 0
Número de missing values en la fila 924 : 0
Número de missing values en la fila 925 : 0
Número de missing values en la fila 926 : 0
Número de missing values en la fila 927 : 0
Número de missing values en la fila 928 : 0
Número de missing values en la fila 929 : 0
Número de missing values en la fila 930 : 0
Número de missing values en la fila 931 : 0
Número de missing values en la fila 932 : 0
Número de missing values en la fila 933 : 0
Número de missing values en la fila 934 : 0
Número de missing values en la fila 935 : 0
Número de missing values en la fila 936 : 0
Número de missing values en la fila 937 : 0
Número de missing values en la fila 938 : 0
Número de missing values en la fila 939 : 0
Número de missing values en la fila 940 : 0
Número de missing values en la fila 941 : 0
Número de missing values en la fila 942 : 0
Número de missing values en la f

...

Número de missing values en la fila 5711 : 0
Número de missing values en la fila 5712 : 0
Número de missing values en la fila 5713 : 0
Número de missing values en la fila 5714 : 0
Número de missing values en la fila 5715 : 0
Número de missing values en la fila 5716 : 0
Número de missing values en la fila 5717 : 0
Número de missing values en la fila 5718 : 0
Número de missing values en la fila 5719 : 0
Número de missing values en la fila 5720 : 0
Número de missing values en la fila 5721 : 0
Número de missing values en la fila 5722 : 0
Número de missing values en la fila 5723 : 0
Número de missing values en la fila 5724 : 0
Número de missing values en la fila 5725 : 0
Número de missing values en la fila 5726 : 0
Número de missing values en la fila 5727 : 0
Número de missing values en la fila 5728 : 0
Número de missing values en la fila 5729 : 0
Número de missing values en la fila 5730 : 0
Número de missing values en la fila 5731 : 0
Número de missing values en la fila 5732 : 0
Número de 

Número de missing values en la fila 5939 : 0
Número de missing values en la fila 5940 : 0
Número de missing values en la fila 5941 : 0
Número de missing values en la fila 5942 : 0
Número de missing values en la fila 5943 : 0
Número de missing values en la fila 5944 : 0
Número de missing values en la fila 5945 : 0
Número de missing values en la fila 5946 : 0
Número de missing values en la fila 5947 : 0
Número de missing values en la fila 5948 : 0
Número de missing values en la fila 5949 : 0
Número de missing values en la fila 5950 : 0
Número de missing values en la fila 5951 : 0
Número de missing values en la fila 5952 : 0
Número de missing values en la fila 5953 : 0
Número de missing values en la fila 5954 : 0
Número de missing values en la fila 5955 : 0
Número de missing values en la fila 5956 : 0
Número de missing values en la fila 5957 : 0
Número de missing values en la fila 5958 : 0
Número de missing values en la fila 5959 : 0
Número de missing values en la fila 5960 : 0
Número de 

Número de missing values en la fila 6180 : 0
Número de missing values en la fila 6181 : 1
Número de missing values en la fila 6182 : 0
Número de missing values en la fila 6183 : 0
Número de missing values en la fila 6184 : 0
Número de missing values en la fila 6185 : 0
Número de missing values en la fila 6186 : 0
Número de missing values en la fila 6187 : 0
Número de missing values en la fila 6188 : 0
Número de missing values en la fila 6189 : 0
Número de missing values en la fila 6190 : 0
Número de missing values en la fila 6191 : 0
Número de missing values en la fila 6192 : 0
Número de missing values en la fila 6193 : 0
Número de missing values en la fila 6194 : 0
Número de missing values en la fila 6195 : 0
Número de missing values en la fila 6196 : 0
Número de missing values en la fila 6197 : 0
Número de missing values en la fila 6198 : 0
Número de missing values en la fila 6199 : 0
Número de missing values en la fila 6200 : 0
Número de missing values en la fila 6201 : 0
Número de 

Número de missing values en la fila 6426 : 0
Número de missing values en la fila 6427 : 0
Número de missing values en la fila 6428 : 0
Número de missing values en la fila 6429 : 0
Número de missing values en la fila 6430 : 0
Número de missing values en la fila 6431 : 0
Número de missing values en la fila 6432 : 0
Número de missing values en la fila 6433 : 0
Número de missing values en la fila 6434 : 0
Número de missing values en la fila 6435 : 0
Número de missing values en la fila 6436 : 0
Número de missing values en la fila 6437 : 0
Número de missing values en la fila 6438 : 0
Número de missing values en la fila 6439 : 0
Número de missing values en la fila 6440 : 0
Número de missing values en la fila 6441 : 0
Número de missing values en la fila 6442 : 0
Número de missing values en la fila 6443 : 0
Número de missing values en la fila 6444 : 0
Número de missing values en la fila 6445 : 0
Número de missing values en la fila 6446 : 0
Número de missing values en la fila 6447 : 0
Número de 

Número de missing values en la fila 6660 : 0
Número de missing values en la fila 6661 : 0
Número de missing values en la fila 6662 : 0
Número de missing values en la fila 6663 : 0
Número de missing values en la fila 6664 : 0
Número de missing values en la fila 6665 : 0
Número de missing values en la fila 6666 : 0
Número de missing values en la fila 6667 : 0
Número de missing values en la fila 6668 : 0
Número de missing values en la fila 6669 : 0
Número de missing values en la fila 6670 : 0
Número de missing values en la fila 6671 : 0
Número de missing values en la fila 6672 : 0
Número de missing values en la fila 6673 : 0
Número de missing values en la fila 6674 : 0
Número de missing values en la fila 6675 : 0
Número de missing values en la fila 6676 : 0
Número de missing values en la fila 6677 : 0
Número de missing values en la fila 6678 : 0
Número de missing values en la fila 6679 : 0
Número de missing values en la fila 6680 : 0
Número de missing values en la fila 6681 : 0
Número de 

In [18]:
# Eliminamos outliers 
for column in df_ba_unido_limpio.columns[1:]:
    #print(df_ba_unido_limpio[column])
    #print(df_ba_unido_limpio[column].dtype)
    if df_ba_unido_limpio[column].dtype== "str":
        continue
    elif df_ba_unido_limpio[column].dtype== "object":
        continue
    else:
        #print(df_ba_unido_limpio[column].dtype)
        q_low = df_ba_unido_limpio[column].quantile(q = 0.01)
        q_hi = df_ba_unido_limpio[column].quantile(q = 0.99)
        f_filtrada = df_ba_unido_limpio[(df_ba_unido_limpio[column]<q_hi) & (df_ba_unido_limpio[column]>q_low)]
f_filtrada

,CODUSU,ANO4_x,TRIMESTRE_x,NRO_HOGAR,REALIZADA,REGION_x,MAS_500_x,AGLOMERADO_x,PONDERA_x,IV1,...,V5_M,V8_M,V9_M,V10_M,V11_M,V12_M,V18_M,V19_AM,V21_M,T_VI
0,TQRMNOQSYHMOTOCDEIJAH00698520,2022,1,1,1,1,S,33,4625,2,...,0,0,0,0,0,0,0,0,0,0
2,TQRMNOPSSHKKMMCDEIIAD00780111,2022,1,1,1,1,S,32,803,1,...,0,0,0,0,0,0,0,0,24000,80000
3,TQRMNORSUHLMNPCDEIIAD00718267,2022,1,1,1,1,S,32,2785,2,...,0,0,0,0,0,0,0,0,0,0
4,TQRMNORSUHLMNPCDEIIAD00718267,2022,1,1,1,1,S,32,2785,2,...,0,0,0,0,0,0,0,0,0,0
5,TQRMNORSUHLMNPCDEIIAD00718267,2022,1,1,1,1,S,32,2785,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6701,TQRMNOPQSHMMKPCDEIJAH00780780,2022,1,1,1,1,S,33,2607,1,...,0,0,0,0,0,0,0,0,0,0
6702,TQRMNOPWPHMLLLCDEIJAH00780781,2022,1,1,1,1,S,33,2325,1,...,0,0,0,0,0,0,0,0,0,28000
6703,TQRMNOPWPHMLLLCDEIJAH00780781,2022,1,1,1,1,S,33,2325,1,...,0,0,0,0,0,0,0,0,0,0
6704,TQRMNORUVHLLKQCDEIJAH00718720,2022,1,1,1,1,S,33,2102,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
for column in f_filtrada.columns[1:]:
    if f_filtrada[column].dtype == "int64":
        continue
    elif f_filtrada[column].dtype == "float64":
        continue
    else:
        print(column, "es una variable", f_filtrada[column].dtype)

MAS_500_x es una variable object
CH05 es una variable object


Chequeamos cuál es el contenido de las variables que nos generarían problemas al no ser numéricas

In [22]:
print(list(f_filtrada.CH05))
print(list(f_filtrada.MAS_500_x))

AttributeError: 'DataFrame' object has no attribute 'CH05'


Observamos que CH05 contiene la fecha de nacimiento de los individuos (con lo cual podríamos borrarla, dado que tenemos su edad en la base también)

MAS_500_x contienen una letra que indica el tamaño del aglomerado. Al habernos quedado con las observaciones de BsAs y Gran BsAs, todas tienen una "S".

Debido a esto, consideramos que podemos deshacernos de tales variables sin mayores problemas.

In [21]:
f_filtrada.drop(columns="CH05", inplace=True)
f_filtrada.drop(columns="MAS_500_x", inplace=True)

C:\Users\lgild\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


KeyError: "['MAS_500_y'] not found in axis"

In [ ]:
print(list(f_filtrada.ITF_y))

Ahora, chequearemos que no queden variables de ingresos o edades con observaciones negativas en nuestra base de datos.

In [ ]:

print(f_filtrada[f_filtrada["ITF"] < 0])
#Obtenemos un dataframe vacío, por lo que no hay observaciones que cumplan con esas características.

print(f_filtrada[f_filtrada["IPCF"] < 0])
#Obtenemos un dataframe vacío, por lo que no hay observaciones que cumplan con esas características.

print(f_filtrada[f_filtrada["P21"] < 0])
#Obtenemos un dataframe con datos de ingreso negativos, por lo que procedemos a eliminarlos.

print(f_filtrada[f_filtrada["P47T"] < 0])
#Obtenemos un dataframe con datos de ingreso negativos, por lo que procedemos a eliminarlos.

print(f_filtrada[f_filtrada["CH06"] < 0])
#Obtenemos un dataframe con datos de edad negativos, por lo que procedemos a eliminarlos.

In [ ]:
# Generamos un loop para asignar los códigos correspondientes (de sexo y edad) a cada observación de la base original
for index, row in df_ba_limpio.iterrows(): 
    if row["CH06"]<1:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V0"
        else: 
            df_ba_limpio.loc[index,"id"]="M0"
    elif row["CH06"]==1:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V1"
        else: 
            df_ba_limpio.loc[index,"id"]="M1"
    elif row["CH06"]==2:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V2"
        else: 
            df_ba_limpio.loc[index,"id"]="M2"
    elif row["CH06"]==3:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V3"
        else: 
            df_ba_limpio.loc[index,"id"]="M3"
    elif row["CH06"]==4:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V4"
        else: 
            df_ba_limpio.loc[index,"id"]="M4"
    elif row["CH06"]==5:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V5"
        else: 
            df_ba_limpio.loc[index,"id"]="M5"
    elif row["CH06"]==6:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V6"
        else: 
            df_ba_limpio.loc[index,"id"]="M6"
    elif row["CH06"]==7:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V7"
        else: 
            df_ba_limpio.loc[index,"id"]="M7"
    elif row["CH06"]==8:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V8"
        else: 
            df_ba_limpio.loc[index,"id"]="M8"
    elif row["CH06"]==9:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V9"
        else: 
            df_ba_limpio.loc[index,"id"]="M9"
    elif row["CH06"]==10:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V10"
        else: 
            df_ba_limpio.loc[index,"id"]="M10"
    elif row["CH06"]==11:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V11"
        else: 
            df_ba_limpio.loc[index,"id"]="M11"
    elif row["CH06"]==12:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V12"
        else: 
            df_ba_limpio.loc[index,"id"]="M12"
    elif row["CH06"]==13:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V13"
        else: 
            df_ba_limpio.loc[index,"id"]="M13"
    elif row["CH06"]==14:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V14"
        else: 
            df_ba_limpio.loc[index,"id"]="M14"
    elif row["CH06"]==15:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V15"
        else:
            df_ba_limpio.loc[index,"id"]="M15"
    elif row["CH06"]==16:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V16"
        else: 
            df_ba_limpio.loc[index,"id"]="M16"
    elif row["CH06"]==17:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V17"
        else: 
            df_ba_limpio.loc[index,"id"]="M17"
    elif (row["CH06"]> 17 and row["CH06"]<30):
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V18"
        else: 
            df_ba_limpio.loc[index,"id"]="M18"
    elif (row["CH06"]>= 30 and row["CH06"]<46):
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V19"
        else: 
            df_ba_limpio.loc[index,"id"]="M19"
    elif (row["CH06"]>= 46 and row["CH06"]<61):
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V20"
        else: 
            df_ba_limpio.loc[index,"id"]="M20"
    elif (row["CH06"]>= 61 and row["CH06"]<75):
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V21"
        else: 
            df_ba_limpio.loc[index,"id"]="M21"
    else:
        if row["CH04"]==1:
            df_ba_limpio.loc[index,"id"]="V22"
        else: 
            df_ba_limpio.loc[index,"id"]="M22"
df_ba_limpio

In [ ]:
auto = pd.read_csv("Auto.csv")

# Ver datos sobre la base en [link](https://rdrr.io/cran/ISLR/man/Auto.html)

# Guardo los vectores de variable dependiente y de variable independiente respectivamente:
y = auto['mpg']
X = auto['horsepower']
X = np.array(X).reshape((-1, 1))

# Parto la base en dos y transformo el vector x: 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)


In [ ]:
# Recuerdan que hemos visto Regresiones Polinomicas, las cuales implican una 
# transformación polinomica de las X, para luego implementar la regresión?

# Veamos un modelo cuadrático:
poly = PolynomialFeatures(degree = 2, include_bias=False) 

# Recordemos en esta instancia setear include_bias a False dado que en la 
# regresión lineal se incluirá la columna de 1s.

#print(x_train)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
  
#print(x_train_poly)

model = LinearRegression().fit(x_train_poly, y_train) 

print('Intercepto:', model.intercept_)
print('Pendiente:', model.coef_)

# Calculamos el Error Cuadrático Medio
y_predpoly = model.predict(x_test_poly)
ecm2 = mean_squared_error(y_predpoly, y_test)
ecm2

In [ ]:
# Veamos un modelo cubico:
poly = PolynomialFeatures(degree = 3, include_bias=False) 

x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
  
model = LinearRegression().fit(x_train_poly, y_train) 
y_predpoly = model.predict(x_test_poly)

ecm3 = mean_squared_error(y_predpoly, y_test)
ecm3

In [ ]:
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
X_seq_poly = poly.fit_transform(X_seq)  

plt.figure()
plt.scatter(x_train, y_train)
plt.plot(X_seq, model.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree 3")
plt.show()

In [ ]:
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
X_seq_poly = poly.fit_transform(X_seq)  

plt.figure()
plt.scatter(x_test, y_test)
plt.plot(X_seq, model.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree 3")
plt.show()

In [ ]:
# Ahora supongamos que cambio la muestra y hago todo lo mismo.
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 500)

# Que error esperarian que obtengamos esta vez?


In [ ]:
# Como podemos repetir el código sin escribirlo por tercera vez?
# Podemos hacer que nuestro código funcione para otros grados?




In [ ]:
ecm2b = transformacion_y_regresion(2, x_train, x_test, y_train, y_test)
ecm3b = transformacion_y_regresion(3, x_train, x_test, y_train, y_test)
ecm4 = transformacion_y_regresion(4, x_train, x_test, y_train, y_test)
ecm5 = transformacion_y_regresion(5, x_train, x_test, y_train, y_test)

In [ ]:
print('Grado2:', ecm2)
print('Grado3:', ecm3)

print('\nGrado2:', ecm2b)
print('Grado3:', ecm3b)
print('Grado4:', ecm4)
print('Grado5:', ecm5)

In [ ]:
# Otra forma de hacer lo mismo pero aprovechando el metodo pipeline:
from sklearn.pipeline import make_pipeline

grado = 3
polyreg = make_pipeline(PolynomialFeatures(grado),LinearRegression())
modelo = polyreg.fit(x_train, y_train)
y_pred_poly = modelo.predict(x_test)

    
# Calculamos el Error Cuadrático Medio
ecm = mean_squared_error(y_pred_poly, y_test)
print(ecm)

###  K-FOLD CROSS-VALIDATION  


In [ ]:
y = auto['mpg']
X = auto['horsepower']
X = np.array(X).reshape((-1, 1))

from sklearn.model_selection import KFold

K = 5

ecms = pd.DataFrame(columns=["grado", "particion", "ecm"])

for grado in range(2, 10):   

    kf = KFold(n_splits=K, shuffle=True, random_state=100)
    
    # El método kf.split aplicado a X nos da los conjuntos de índices que necesitamos para
    # partir nuestros conjunto de datos en training y testing en cada iteración.
    #  OXXXX
    #  XOXXX
    #  XXOXX
    #  XXXOX
    #  XXXXO
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):   
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        ecm, modelo = transformacion_y_regresion(grado, x_train, x_test, y_train, y_test)
            
        ecms = ecms.append({"grado": grado, "particion": i, "ecm": ecm}, ignore_index=True)            
    
ecms = ecms.astype({"grado":int, "particion":int})
ecms

In [ ]:
# Otra opción podría ser visualizandolo en un boxplot
import seaborn as sns
sns.set()
ss = sns.boxplot(data=mses, x="grado", y="mse")


In [ ]:
#Una opción para ver el mejor modelo sería sacar el error promedio para cada grado:
ecms.groupby('grado').agg({'ecm':'mean'})

In [ ]:
#Construyan una función que les permita seleccionar 


In [ ]:
#Fianalmente construimos el modelo polinomial de grado 6 y lo graficamos 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

ecm, modelo = transformacion_y_regresion(grado, x_train, x_test, y_train, y_test)
        
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
poly = PolynomialFeatures(degree = grado, include_bias=False) 
X_seq_poly = poly.fit_transform(X_seq)  

plt.figure()
plt.scatter(x_train, y_train)
plt.plot(X_seq, modelo.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree {}".format(grado))
plt.show()
